In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np 
import pandas as pd
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras import layers
import os

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
from tensorflow.keras.applications import ResNet50

In [4]:
keras.backend.clear_session()

In [5]:
model = ResNet50(weights = 'imagenet', include_top = False, input_shape = (32,32,3))

In [6]:
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 16, 16, 64)   256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [7]:
model.trainable = False

In [8]:
model1 = model.output
model1 = Flatten()(model1)
model1 = Dense(1000, activation='relu', name = 'd1', trainable = True)(model1)
model1 = Dense(500, activation='relu', name = 'd2', trainable = True)(model1)
model1 = Dense(250, activation = 'relu', name ='d3', trainable = True)(model1)
model1 = Dense(125, activation='relu', name = 'd4', trainable = True)(model1)
model1 = Dense(62, activation = 'relu', name ='d5', trainable = True)(model1)
model1 = Dense(10, activation='softmax', name ='preds', trainable = True)(model1)

In [9]:
model1.trainable = True

In [10]:
modelf = Model(inputs = model.input, outputs = model1)

In [11]:
modelf.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 16, 16, 64)   256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [12]:
images_path = r'C:\Users\offic\Documents\GitStuff\NOTEMNIST\notMNIST_small\notMNIST_small'

In [13]:
classes = os.listdir(images_path)

In [14]:
classes


['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']

In [15]:
datagen = ImageDataGenerator(images_path,
                             zoom_range = [0.6, 1.9], 
                             width_shift_range=0.3, 
                             height_shift_range=0.3,
                             fill_mode='constant', 
                             cval=190,
                             rescale=1.0/255.0,
                             )

In [16]:
train_gen = datagen.flow_from_directory(images_path,
        target_size=(32, 32),
        batch_size=32,
        subset = 'training',
        classes = classes
        )

Found 18724 images belonging to 10 classes.


In [17]:
test_gen = datagen.flow_from_directory(images_path,
        target_size=(32, 32),
        batch_size=32,
        subset = 'validation',
        classes = classes
        )

Found 0 images belonging to 10 classes.


In [18]:
train_steps = train_gen.samples // 32
val_steps = test_gen.samples // 32

In [19]:
modelf.compile(optimizer= 'adam', loss='categorical_crossentropy', metrics = ['accuracy'])

In [20]:
es = EarlyStopping(monitor = 'val_loss',
                   patience = 5,
                   mode = 'auto'
                   )
reduce_lr = ReduceLROnPlateau(monitor = 'val_loss',
                               patience = 2,
                              mode = 'auto'
                              )

checkpoint_filepath = r'C:\Users\offic\Documents\GitStuff\NOTEMNIST\Checkpoint resnet50small'
ckpts = tf.keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_filepath,
    save_weights_only = False,
    monitor = 'val_loss',
    mode = 'auto',
    save_best_only = True)


In [21]:
modelf.fit(train_gen,
           epochs = 500,
           validation_data = test_gen,
           steps_per_epoch = train_steps,
           validation_steps = val_steps,
           callbacks = [es, reduce_lr, ckpts] 
           )

Epoch 1/500

c:\users\offic\venv\lib\site-packages\keras_preprocessing\image\image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node model/conv1_conv/Conv2D (defined at <ipython-input-21-934bec236b0a>:1) ]] [Op:__inference_train_function_12180]

Function call stack:
train_function
